# First Assigment  FHPC course 


##Section 0: (warm-up)

- Compute Theoretical Peak performance for your laptop/desktop
  - Identify the CPU                       

  - Identify the frequency       

  - Identify how many cores   

  ​            Suggestion for Linux User:  `cat /proc/cpuinfo`  gives you all the information you neeed.

  - Identify the number of floating point for cycle  
    
      This is is tricky: try to find out some info about the global performance looking here:
      - https://en.wikipedia.org/wiki/FLOPS
      - https://stackoverflow.com/questions/23588380/how-to-compute-the-single-precision-data-and-double-precision-data-peak-performa?rq=1
  - Write here the  final result  obtained: 
  

  |        | Your model | CPU  | Frequency | number  of core | Peak performance |
  | ------ | ---------- | ---- | --------- | --------------- | ---------------- |
  | Laptop |            |      |           |                 |                  |


 

- Compute sustained Peak performance for your  cell-phone
  
  - Identify an app to run HPL, for instance:  https://play.google.com/store/apps/details?id=com.sqi.linpackbenchmark
  
  - Run&Tune it to get the best number you can 
  
    Suggestion: try to increase the size of the matrix
  
  - Extra Bonus:  Identify the CPU and calculate the Peak performance of the CPU  
  
  
  |            | Your model | sustained performance | size of matrix | Peak performance | memory |
  | ---------- | ---------- | --------------------- | -------------- | ---------------- | ------ |
  | SmartPhone |            |                       |                |                  |        |



- Find out in which year  your cell phone/laptop  could have been in top1% of Top500 

  - Fill in the following table

    

  |            | Your model | performance | Top500 year | number 1 HPC system | number of processors (TOP500) |
  | ---------- | ---------- | ----------- | ----------- | ------------------- | ----------------------------- |
  | SmartPhone |            |             |             |                     |                               |
  | Laptop     |            |             |             |                     |                               |


​    
## Section  1: theoretical model

_I've decided to leave the premises for this section and to answer to the direct questions asked directly below it to have a clearer layout._

- devise a performance model for a simple parallel algorithm: sum of N numbers

  - Serial Algorithm : n-1 operations 

    $T_{serial}= N*T_{comp}$ 
    $T_{comp}$= *time to compute a floating point operation*

  - Parallel Algorithm : master-slave	

    - read N and distribute N to P-1  slaves ===>  $T_{read} + (P-1)\times T_{comm}$ 
      $T_{comm}$ = *time  each processor takes to communicate one message, i.e. latency..*
      $T_{read}$   = *time  master takes to read* 

    - N/P sum over each processors (including master)  ===> $T_{comp}/N$

    - Slaves send partial sum  ===>   $(P-1)T_{comm}$

    - Master performs  one final sum ===>  $(P-1) \times $T_{comp}$

      the final model:    $T_p=   T_{comp}\times (P -1 + n/P)  + T_{read} + 2(P-1)\times T_{comm}  $

- compute scalability curves for such algorithm and make some plots

  - assumptions:

    - $T_{comp} =2 \times 10^{-9}$
    - $T_{read}= 1 \times 10^{-4}$
    - $T_{comm}= 1 \times 10^{-6}$

    Play with some value of N and plot against P  (with P ranging from 1 to 1000)
    
  ![TheoreticalPartialSumSpeedup](img/Theoretical_speedup_partial_sum.png)

- Comment on them:

  **- For which values of N do you see the algorithm scaling ?**
      
   The algorithm should start scaling for a low number of processors for N as low as $10^5$. We can really see a dramatic improvement in scaling for $N = 10^8$ This is due to the fact that the $T_{comp}$ factor is $10^3$ lower than the $T_{comm}$ factor, therefore we need a large $ n/P$ addend to scale well the algorithm.

  **- For which values of P does the algorithm produce the best results ?**
  
    #TODO

  **- Can you try to modify the algorithm sketched above to increase its scalability ? **
  
  Yes! We can for example redesign both the communication from the master to the slaves and viceversa in this way:
  
  - Instead of communicating N to P-1 slaves, we could send the information just to $\sqrt{P-1}$ slaves and then make each one of them pass the same information (i.e. N) to $\sqrt{P-1} - 1$ slaves. This way our initial communication term will be equal to = $2 \times \sqrt{P-1} - 1$ .
  
  - The same thing can be done for the inverse path. A collection of different $\sqrt{P-1} - 1$ slaves all send their partial sum to their respective "sub-master" simultaneously. Every $\sqrt{P-1}$ sub-master computes the partial sum and then send the result to the master repeating this very same process for each submaster.
  
  This way our total communication term will be reduced to $(4 \times \sqrt{P-1} - 2) \times T_{comm}$ and our computational term will be $(2 \times \sqrt{P-1} -2 + n/P) \times T_{comp}$.
  
  Final corrected model:  $T_p=   T_{comp}\times ( 2 \times \sqrt{P-1} -2 + n/P)  + T_{read} + (4 \times \sqrt{P-1} - 2)\times T_{comm}  $
      
  

## Section 2 : play with MPI program

### 2.1:  compute strong scalability of a mpi_pi.c program

In this section we will briefly describe the results of the analysis of the different execution time yielded by the code provided in class to compute PI using Monte-Carlo integration. The result refer both to the execution of the serial algorithm and to the parallel one.

#### Time measurement considerations

Firstly, it needs to be addressed the problem regarding the measure of time of an execution of a code. To measure the execution time of the serial algorithm, the idea is to use the command /usr/bin/time, as suggested by the assignment. This standard shell command yields three different measures which names are real, usr and sys. (#TODO Maybe a brief explanation of the three times). For a serial application, as in our case, we'd expect usr+sys to be approximatively equal to real, but, since we don't want to take into consideration the system time as per request of the assignment, we'll just use the usr time, i.e. the time employed by the cpu to run our program. 




  |              |  n=100000  |  n=1000000 |  n=10000000 |  n=100000000 |  n=1000000000 |  
  | ----------   | ---------- | ---------  |  ---------- |  ----------  |  ----------   | 
  | user time(s) |    0.003   |    0.021   |    0.197    |    1.966     |   19.665      |
  
_User time captured for the execution of the serial algorithm. Keeping into account some truncation (#TODO: maybe  avoid them) we can see that the user time scales linearly with n_
 

To identify the overhead of MPI routines, we can just consider the difference in between the usr time returned by the serial program, and the one returned by the parallel program using just one processor; since the computational part is identical in this case, this will yield us a rough estimate of the time involved in the communication, initialization and finalization routines.



  |                    |  n=100000  |  n=1000000 |  n=10000000 |  n=100000000 |  n=1000000000 |  
  | ----------         | ---------- | ---------  |  ---------- |  ----------  |  ----------   | 
  | user serial(s)     |    0.003   |    0.021   |    0.197    |    1.966     |   19.665      |
  | user paral(s), P=1 |    1.147   |    1.172   |    1.342    |    3.124     |   21.011      |
  | Difference         |    1.144   |    1.151   |    1.145    |    1.158     |    1.346      |
_User time captured both for the serial algorithm and for the parallel algorithm with one processor only. The difference between them remains pretty much constant with respect to n._



This approach cannot be valid also for our parallel code when we run it with multiple processor though, since the user time takes into consideration the sum of the times for every CPU. The only way in which we can actually have an extimation of the execution time of our parallel program (from now on denoted as T(P)) is to use the walltime provided by the MPI_Wtime calls inside the code. Of all the printed walltimes, our choice is to take the maximum one, since this is the measure of time of the processor that runs the longest. It is worth noting that this processor is not necessarily going to be the master, because of the way the communication are defined in the code. Since the choice of the programmer was to use a SSend routine in the code, we know from the theory that the last processor to send the information to the master will also be the slowest, up until a large enough (roughly $10^3$) number of processors. This is due to the fact that this processor needs to wait for an answer from the master, confirming the actual reception of the message. Since the time of a single communication is roughly 10^3 slower than the time of a single computation, the master will finish doing the last sum while its confirmation message to the last sending processor is still being delivered. 


### Strong scaling

In order to verify the strong scaling of our code, we need to plot the different runtime values of our code against the number of processors used. We will acquire the data in the way that we've described in the previous paragraph, considering the maximum walltime out of all the processors for the previously mentioned reasons. We will also plot these data for 4 different values of N, increasing N by a factor of 10 each time; in this way it will be clear which is the size of N for which we can say that our code scales strongly.

///#TODO why is it showing this image???
![SScalingMontecarloPi](img/StrongScalingMontecarloPi.png)

The image shows for different values of N the value of the execution time for increasing P. As can be seen from the plots, for every N our curve has a tendency to increase with respect to P, and this tendency becomes increasingly linear the higher the N value. This shows that the mpi_pi algorithm tends to scale strongly for a large N, at least with regards to a relatively (to N) small number of processors. In particular, we can say that for $N = 10^6$ our code stops to scale strongly for $P \approx bla$ 


### 2.2: identify a model for the parallel overhead

Based on the data we have already collected, we can estimate the parallel overhead of our algorithm by plotting the speedup obtained and already commented on the previous paragraph, with the perfect speedup, i.e $Speedup(P) = P$. The difference of the two functions is exactly what we call the parallel overhead of the program, i.e. all the time that the processes spend computing something not directly related to our algorithm (most of it it's due to latency in between communications, which is unavoidable to a certain extent).

![ParallelOverhead](img/pMinusTrueSpeedup.png)

The difference in between the orange curve (perfect speedup) and the blue curve (actual speedup) gives us an idea of the parallel overhead of our code. As for the speedup commented in the previous paragraph, we can see that for $N=10^6$ the parallel overhead start increasing almost linearly after $\approx$ 10 processors. This gives us an idea that the latency in between the communication from the nodes to the master is predominant with respect to the computation time for low values of N, and gets negligible for $N=10^9$ for the number of P considered.


### 2.3: weak scaling 

Doing a weak scaling test means to consider the execution time of an algorithm over a varying number of processor so that $N/P$ remains constant. Therefore, I've decided to choose $N_{0}=10^7, 10^8, 10^9$ as the starting value for my test and then increase N proportionally with the number of P. What we would expect for a perfectly weak scaling code is a constant line with respect to the P axes, because our code is computing exactly $N/P$ operations which is what is being kept constant. In practice, as it was for the strong scaling consideration, this does not take into consideration a latency effect in communication which is actually affecting our code. Since the number of communication is increasing with respect to the increase of P, our code will tend to increase its runtime by increasing the number of processors. This tendency will be smaller for a low number of P and it will tend to increase for a large number of P, when the latency factor increases with respect to the computation time.
Conversely the SpeedUp should be increasing as P since our serial code execution time is directly proportional to N, but this is not the case for higher values of P for all I've just described.


Here are the results in terms of T(P) and Speedup(P). The Speedup plot is on the left while the execution time plot is on the right.


![WeakScalingPi](img/WeakScalingMontePi.png)



The tendencies of both Speedup(P) and T(P) are close to what we actually described previously in the paragraph. We can also notice from these data how the random variation of execution time increases for higher values of $N/P$.


## Section 3: implement a parallel program using MPI 

We chose to write the program in C, since it's the language we've been using the most during these first hpc lectures. The code implements an algorithm that follows the model discussed during the lectures. In particular, it performs the following steps:

* Defines a master node that reads from command-line argument the number N of which we want to calculate the partial sum.

* Sends to P-1 slaves processor the number N/P. In particular, I've decided to communicate to every slave processor the number N, and make them perform two more calculation: n/p and n\%p. I've done this to solve the problem of redestribution of the remainder of N/P. In fact, calculating the remainder inside the master node and then splitting it to every slave node would have required another communication, thus increasing the actual performance of another P*T_comm, which is 10^3 times larger than P*T_comp. After calculating n\%p, every slave processor allocates a memory space one unit greater than the others, if its rank is smaller than the remainder obtained.

* Computes all the partial sum computation in every processor, master included

* Sends the partial results to the master which receives them and sums together the last P numbers

As described in point 2, the algorithm performs two operations per processor more than what we defined in section 1, this is why I have updated the theoretical code in the first section. /////#TODO Actually remember to do it

The code runs perfectly up until $N \approx 10^{10} $ where the number to be calculated gets too high and the program overflows, despite having defined the largest possible datatype in the source code.

Finally, we wrote a second version of the code using the collective MPI operations such as MPI_Broadcast and MPI_Reduce, instead of having to specify both Send and Receive for every communication.

More technical information regarding the usage of the code will be given in the readme file provided.



## Section 4  (run and compare)

In this section we present the result of the execution of the program built in section 3. Just like in section 2, we present plots regarding strong scaling and weak scaling test on the algorithm and we comment on them.

###Strong Scaling

We'll first start by investigating if our implementation of the algorithm presents a strong scalability by making the same plots produced for the Montecarlo Pi algorithm. We'll both plot the strong scalability calculated by taking the maximum 

![StrongScalingSum](img/SumStrongScaling.png)

The plots here show a very poor tendency towards strong scalability. In particular, the plot with $N=10^6$ doesn't show a defined tendency already from $P \approx 6$ and also the plot for $N=10^7$ is already started slowing down towards the high value of P considered. Both the plot for $N=10^8$ and $N=10^9$ show an increasing trend, but the absolute value of the speedup remains pretty low compared to the perfect speedup, hinting towards the fact that there is a high parallel overhead present in our code even with a few number of processors. Since we know that latency overhead scales linearly with P, we can probably say that this is what is causing the speedup curve to slow down for $N=10^6$ and $N=10^7$, but this cannot be the reason why the speedup is absolutely low. In fact, this is due to an implementation difference in between the serial code and the parallel code: in the first one we don't allocate an array on which we perform the sum, while in the parallel one we do. This slows down the code proportionally to n and it's exactly what we are experiencing in these plots.


![PracticalVSTheoreticalSU](img/Practical_Theoretical_sum_speedup.png)

The assumption made before about the slow speedup are strenghtened by the comparison in between our theoretical model and the actual speedup computed using $T_{read}$, $T_{comm}$ and $T_{comp}$. Here we can see a similar behaviour to the previous plot, looking at the trend of the curves, since both are showing a poor increase for $N=10^6$ and $N=10^7$ and a good strong scaling trend for $N=10^8$ and $N=10^9$. However, here the speedup absolute value is much closer to its perfect value compared to the previous measure, and this is because the times here considered were taken without considering the array values allocation. This results in a much better representation of scalability as a whole, even though it is probably unbalanced for little values of P, as we can observe some phenomena of super-linearity in our measured scalability.

Nevertheless, a general trend highlighted by both plots, is that we have somehow underestimated the impact of communication in our theoretical model, as every graph for high values of P is way below its estimated value. This might mean that the actual $T_{comm}$ value for one single communication is much higher than expected, or that the number of communication occuring between the master and the slaves is a multiple of what we thought (perhaps, just like SSend, also Send needs a confirmation of reception, in which case the dependency from P would be doubled).


###Weak Scaling

In this final section you are requested to run and measure strong scalability of  the program written in section 3.



  - Run the implemented program for different large enough sizes of the array 

  - Plot as in section 3 scalability of the program  

  - compare  performance results obtained against the performance model elaborated in section 2

      - comment on the assumption made in section 1 about times: are they correct ? do you observe something different ? 

        




3